# 复习内容

In [1]:
import numpy as np

#创建实验样本，
def loadDataSet():
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]    #1 代表侮辱性的词汇，0不是，对应上面的语句列表
    return postingList, classVec

In [2]:
#创建一个包含在文档中中不存在重复的词的列表
def createVocabList(dataSet):
    vocabSet = set([])  #create empty set
    for document in dataSet:
        vocabSet = vocabSet | set(document) # 两个集合的并集
    #vocabSet为一个不重复此表
    return list(vocabSet) #返回转型为list

#word2vec输入参数为词汇表，和一个文档
#返回的文档向量，对应词汇表中的单词在输入文档中是否出现，创建一个相对应的向量
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0 for i in range(len(vocabList))]
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else: 
            print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

In [3]:
listOPosts, ListClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)

In [4]:
listOPosts[0]

['my', 'dog', 'has', 'flea', 'problems', 'help', 'please']

In [5]:
setOfWords2Vec(myVocabList, listOPosts[0])
#对应listOPosts[0]文本特征为一行[1, vocabsize]的向量
# vocabsize相当于特征

[0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1]

In [6]:
#训练并计算概率
def trainNB0(trainMatrix, trainCategory):
    #trainMatrix——x
    #trainCategory——y
    numTrainDocs = len(trainMatrix) #训练数据数量
    numWords = len(trainMatrix[0]) #数据词向量长度，特征数量
    pAbusive = sum(trainCategory)/float(numTrainDocs) #正类别数量/样本总数
    #同时作为出现正样本的概率
    
    p0Num = np.zeros(numWords); 
    p1Num = np.zeros(numWords)     #初始化概率计算的分子，由多少个词即有多少个
    
    p0Denom = 0
    p1Denom = 0                        #分母变量初始化，此处加入初值2，作为平滑处理的值防止出现bug
    
    #统计对应正负样本各个属性特征的数量之和
    #统计正负样本词综述
    for i in range(numTrainDocs): #遍历每一个样本呢
        if trainCategory[i] == 1: #如果对应分类=1
            p1Num += trainMatrix[i] #对应词向量相加为相对应类的和
            p1Denom += sum(trainMatrix[i]) #获取正样本总词语数量
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])#获取负样本总词语数量
    p1Vect = (p1Num/p1Denom)          #当样本为正样本部分，n为词向量每个出现的概率
    p0Vect = (p0Num/p0Denom)          #返回对应样本出现的概率
    #即为计算在输出为正的情况下，出现各类属性特征的概率
    
    return p0Vect, p1Vect, pAbusive #返回两个向量，一个概率（正样本概率），二分类可求负样本概率

In [7]:
listOPosts, listClasses = loadDataSet()
trainMat = []
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))

In [8]:
p0V, p1V, pAb = trainNB0(trainMat, listClasses)

In [9]:
p0V, p1V, pAb

(array([0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.        , 0.        , 0.04166667, 0.04166667, 0.        ,
        0.        , 0.        , 0.04166667, 0.04166667, 0.        ,
        0.        , 0.        , 0.04166667, 0.04166667, 0.        ,
        0.08333333, 0.04166667, 0.        , 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.        , 0.04166667,
        0.125     , 0.04166667]),
 array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.05263158, 0.05263158, 0.        , 0.        , 0.05263158,
        0.05263158, 0.05263158, 0.        , 0.        , 0.05263158,
        0.05263158, 0.05263158, 0.        , 0.        , 0.05263158,
        0.05263158, 0.        , 0.10526316, 0.        , 0.05263158,
        0.        , 0.        , 0.05263158, 0.15789474, 0.10526316,
        0.        , 0.        ]),
 0.5)

In [10]:
# 朴素贝叶斯分类函数

# 分类函数
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)    #理论上求概率与类别相乘，由于p1vec即为log输出结果，因此与类别相乘改为相加
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)#其实这里时概率相乘，但是因为改成log所以在此时在可以进行相加，条件概率假设独立
    #求取在
    if p1 > p0:
        return 1
    else:
        return 0

In [11]:
# 分类函数

listOPosts, listClasses = loadDataSet()#生成样例数据
myVocabList = createVocabList(listOPosts) #创建词向量列表
trainMat = [] #初始化训练矩阵
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc)) 
#生成训练集矩阵
p0V, p1V, pAb = trainNB0(np.array(trainMat), np.array(listClasses))

In [12]:
# 返回输出预测值
testEntry = ['love', 'my', 'dalmation']
thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))#生成词向量

classifyNB(thisDoc, p0V, p1V, pAb)

0

# 文本向量处理

In [1]:
import numpy as np
import random 
with open("cnews/cnews.train.txt", 'r', encoding="utf-8") as file:
    train = file.readlines()
with open("cnews/cnews.test.txt", 'r', encoding="utf-8") as file:
    test = file.readlines()

In [2]:
def exactua(data):
    random.shuffle(data) #数据集打乱
    x_data = []
    y_data = []
    for line in data:
        x_data.append(line.replace('\n', '').split('\t')[1])
        y_data.append(line.replace('\n', '').split('\t')[0])
    return x_data, y_data

In [3]:
x_train_data, y_train_data = exactua(train)
x_test_data, y_test_data = exactua(test)

In [4]:
# 1000条 训练集
# 100条 测试
x_train_data, y_train_data = x_train_data[:1000], y_train_data[:1000]
x_test_data, y_test_data= x_test_data[:100], y_test_data[:100]

In [5]:
import jieba 

def stopwordslist():
    stopwords = [line.strip() for line in open('stopwords.txt',encoding='UTF-8').readlines()]
    return stopwords

In [6]:
stopwords = stopwordslist()

In [7]:
def token(x_train_data, x_test_data):
    #去停用词并进行结巴分词
    trainlists = []
    for i in x_train_data:
        word_list = [word for word in jieba.cut(i) if word not in stopwords]
        trainlists.append(' '.join(word_list))
    testlists = []
    for i in x_test_data:
        word_list = [word for word in jieba.cut(i) if word not in stopwords]
        testlists.append(' '.join(word_list))
    return trainlists, testlists

In [8]:
x_train, x_test = token(x_train_data, x_test_data)
len(x_train)
len(x_test)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\wttree\AppData\Local\Temp\jieba.cache
Loading model cost 0.815 seconds.
Prefix dict has been built succesfully.


100

In [9]:
def data2vec(x_train, x_test):
    from sklearn.feature_extraction.text import CountVectorizer

    count_vec=CountVectorizer(stop_words=stopwords) #创建词袋数据结构
    X_count_train = count_vec.fit_transform(x_train)  #<class 'scipy.sparse.csr.csr_matrix'>
    #train和test转化为特征向量
    X_count_train= X_count_train.toarray()
    X_count_test = count_vec.transform(x_test).toarray()   
    return X_count_train, X_count_test, dict(count_vec.vocabulary_)

In [10]:
x_train, x_test, wordDic = data2vec(x_train, x_test)

C:\software\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [11]:
x_train.shape

(1000, 40128)

In [12]:
print ('vocabulary:\n\n')
count = 0 
for key,value in wordDic.items():
    print(key,value)
    count += 1
    if count > 10:
        break

vocabulary:


港交所 26792
地产股 13671
加速 10174
分裂 9480
富力 16402
地产 13668
风向标 39140
理财 28254
周报 12665
记者 34428
张伟 18393


In [13]:
def y2label(y_train, y_test):
    dic = {}
    count = 0
    for i in set(y_train):
        dic[i] = count 
        count += 1
    y_train, y_test = [dic.get(i) for i in y_train_data], [dic.get(i) for i in y_test_data]
    return y_train, y_test, dic

In [14]:
y_train, y_test, dicLabel = y2label(y_train_data, y_test_data)

In [16]:
y_train[:3]

[6, 3, 2]

## 贝叶斯分类器

In [56]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha = 0.01).fit(x_train,y_train)#


MultinomialNB（alpha = 1.0，fit_prior = True，class_prior = None)  
Parameters:	   
alpha : float, optional (default=1.0)  
Additive (Laplace/Lidstone) smoothing parameter (0 for no smoothing).  
  
fit_prior : boolean, optional (default=True)  
Whether to learn class prior probabilities or not. If false, a uniform prior will be used.  

class_prior : array-like, size (n_classes,), optional (default=None)  
Prior probabilities of the classes. If specified the priors are not adjusted according to the data.

In [57]:
#  训练集测试
result = clf.predict(x_train)
count = 0
for i in range(len(result)):
    if result[i] == y_train[i]:
        count += 1
print(count/len(x_train))

0.999


In [58]:
#  测试集测试
result = clf.predict(x_test)
count = 0
for i in range(len(result)):
    if result[i] == y_test[i]:
        count += 1
print(count/len(x_test))

0.87


# 结合Tf-idf 算法进行文本分类

In [105]:
import numpy as np
import random 
with open("cnews/cnews.train.txt", 'r', encoding="utf-8") as file:
    train = file.readlines()
with open("cnews/cnews.test.txt", 'r', encoding="utf-8") as file:
    test = file.readlines()
x_train_data, y_train_data = exactua(train)
x_test_data, y_test_data = exactua(test)
# 1000条 训练集
# 100条 测试
x_train_data, y_train_data = x_train_data[:1000], y_train_data[:1000]
x_test_data, y_test_data= x_test_data[:100], y_test_data[:100]
x_train, x_test = token(x_train_data, x_test_data)
y_train, y_test, dicLabel = y2label(y_train_data, y_test_data)

In [106]:
def data2vec(x_train, x_test):
    from sklearn.feature_extraction.text import TfidfTransformer 
    from sklearn.feature_extraction.text import CountVectorizer 
    
    count_vec=CountVectorizer(stop_words=stopwords) #创建词袋数据结构
    transformer = TfidfTransformer()
    X_count_train = count_vec.fit_transform(x_train)
    X_count_test = count_vec.transform(x_test)
    
    X_train_tfidf = transformer.fit_transform(X_count_train).toarray()
    X_test_tfidf = transformer.fit_transform(X_count_test).toarray()
    return X_train_tfidf, X_test_tfidf

In [118]:
x_train, x_test = data2vec(x_train, x_test)

C:\software\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [119]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha = 1).fit(x_train,y_train)#


In [120]:
#  训练集测试
result = clf.predict(x_train)
count = 0
for i in range(len(result)):
    if result[i] == y_train[i]:
        count += 1
print(count/len(x_train))

0.992


In [121]:
#  测试集测试
result = clf.predict(x_test)
count = 0
for i in range(len(result)):
    if result[i] == y_test[i]:
        count += 1
print(count/len(x_test))

0.89
